3.3.3. A plate with notches  

预测应力 ${\sigma_{eff}}$

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from mxnet import autograd, gluon, init, np, npx
from mxnet.gluon import nn
from d2l import mxnet as d2l
import random
import numpy
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.tri as mtri
from mpl_toolkits.axes_grid1 import make_axes_locatable

npx.set_np()

In [ ]:
DXX=pd.read_table(r"..\Data\NN140_LBR_Ev_Frand_XX.txt", sep='\s+', header=None)
DYY=pd.read_table(r"..\Data\NN140_LBR_Ev_Frand_YY.txt", sep='\s+', header=None)

In [ ]:
NXX=np.array(DXX.values, dtype=np.float32, ctx=npx.gpu()).reshape(10000,8,1,140)
YY=DYY.iloc[:,140*5:140*6]

In [ ]:
n_train = 8000
train_features = NXX[:n_train,:,:,:]
test_features = NXX[n_train:,:,:,:]
train_labels = np.array(YY[:n_train].values, dtype=np.float32, ctx=npx.gpu())
test_labels = np.array(YY[n_train:].values, dtype=np.float32, ctx=npx.gpu())

In [ ]:
batch_size=128
train_iter = d2l.load_array((train_features, train_labels), batch_size)
test_iter = d2l.load_array((test_features, test_labels), batch_size)

In [ ]:
def MSE(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += ((net(X[:,[0,1,2,3,6,7],:,:],X[:,[4,5],:,:])-y)**2).sum()
        n += y.size
    return acc_sum / n

In [ ]:
class NetFEM(nn.Block):
    def __init__(self):
        super().__init__()
        self.Batch1=nn.BatchNorm()
        self.Batch2=nn.BatchNorm()
        self.Conv2D1=nn.Conv2D(channels=4, kernel_size=3, padding=1, activation='relu')
        self.Dens1X1=nn.Dense(280, activation='relu')
        self.Dens2X1=nn.Dense(140)
        self.Dens1X2=nn.Dense(280, activation='relu')
        self.Dens2X2=nn.Dense(140)
        self.output=nn.Dense(140)

    def forward(self, X1, X2):
        X1=self.Batch1(X1)
        X1=self.Conv2D1(X1)
        X1=self.Dens1X1(X1)
        X1=self.Dens2X1(X1)
        X2=self.Batch2(X2)
        X2=self.Dens1X2(X2)
        X2=self.Dens2X2(X2)
        X=X1*X2
        X=self.output(X)
        return X

In [ ]:
net = NetFEM()
net.initialize(init.Normal(sigma=0.01), ctx=npx.gpu())
loss = gluon.loss.L2Loss()

In [ ]:
train_ls, test_ls = [], []

In [ ]:
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': 0.001,})

In [ ]:
old_epochs=0
new_epochs=50

for epoch in range(new_epochs):
    for X, y in train_iter:
        with autograd.record():
            l = loss(net(X[:,[0,1,2,3,6,7],:,:],X[:,[4,5],:,:]), y)
        l.backward()
        trainer.step(batch_size)
    train_ls.append(MSE(train_iter, net))
    test_ls.append(MSE(test_iter, net))
    print('epoch %d, train MSE %.6f, test MSE %.6f' % (old_epochs+epoch+1, train_ls[-1], test_ls[-1]))

In [ ]:
plt.plot(np.arange(old_epochs+new_epochs),train_ls,label='train data')
plt.plot(np.arange(old_epochs+new_epochs),test_ls,label='test data')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.yscale('log')
plt.legend()

In [ ]:
# net.save_parameters("2DelasticPlus_LWREv_0__Batch128_1+8")

In [ ]:
def plots(net,ax,iindex=0):
    if iindex==0:
        ii=random.randint(n_train,DXX.shape[0])
    else:
        ii=iindex
    xd=DXX.iloc[ii,0:140].values
    yd=DXX.iloc[ii,140:280].values
    px=NXX[ii,:,:,:]
    py=YY.iloc[ii].values
    
    nxx=px.reshape(1,8,1,140)
    nyy=net(nxx[:,[0,1,2,3,6,7],:,:],nxx[:,[4,5],:,:]).reshape(140,)
    
    zd=py
    zn=nyy.asnumpy()

    ax.scatter3D(xd,yd,zd, color='r',label='FEM' )
    ax.scatter3D(xd,yd,zn, color='b',label='NET' )
    ax.legend()

def plotopo(net,fig,style=1,iindex=0):
    if iindex==0:
        ii=random.randint(n_train,DXX.shape[0])
    else:
        ii=iindex
    xd=DXX.iloc[ii,0:140].values
    yd=DXX.iloc[ii,140:280].values
    rr=yd[0]
    triang = mtri.Triangulation(xd, yd)
    xmid = xd[triang.triangles].mean(axis=1)
    ymid = yd[triang.triangles].mean(axis=1)
    mask = numpy.where(xmid**2 + ymid**2 < rr**2, 1, 0)
    triang.set_mask(mask)
    
    px=NXX[ii,:,:,:]
    py=YY.loc[ii].values
    
    nxx=px.reshape(1,8,1,140)
    nyy=net(nxx[:,[0,1,2,3,6,7],:,:],nxx[:,[4,5],:,:]).reshape(140,)
    
    zd=py
    zn=nyy.asnumpy()
    
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.set_aspect(1)
    if style==1:
        im=ax1.tripcolor(triang, zd, cmap="jet", shading='gouraud')
    elif style==2:
        im=ax1.tricontourf(triang, zd, cmap="jet")
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="6%", pad=0.15)
    cbar = plt.colorbar(im, cax=cax)
    # cbar.set_label('$u_y$',size=10)
    cbar.ax.tick_params(labelsize=10)
    ax1.set_title('FEM')

    ax2 = fig.add_subplot(1, 2, 2)
    ax2.set_aspect(1)
    if style==1:
        im=ax2.tripcolor(triang, zn, cmap="jet", shading='gouraud')
    elif style==2:
        im=ax2.tricontourf(triang, zn, cmap="jet")
    divider = make_axes_locatable(ax2)
    cax = divider.append_axes("right", size="6%", pad=0.15)
    cbar = plt.colorbar(im, cax=cax)
    # cbar.set_label('$u_y$',size=10)
    cbar.ax.tick_params(labelsize=10)
    ax2.set_title('NET')


In [ ]:
fig=plt.figure(figsize=(10,8))
ax = fig.add_subplot(2, 2, 1, projection='3d');plots(net,ax)
ax = fig.add_subplot(2, 2, 2, projection='3d');plots(net,ax)
ax = fig.add_subplot(2, 2, 3, projection='3d');plots(net,ax)
ax = fig.add_subplot(2, 2, 4, projection='3d');plots(net,ax)

In [ ]:
for i in range(3):
    fig=plt.figure(figsize=(4,4));plotopo(net,fig,1)

In [ ]:
fig=plt.figure(figsize=(10,10));plotopo(net,fig,2)